In [0]:
# =====================================================
# ZADANIE 1 + 2: WALIDACJA DŁUGOŚCI + ANALIZA JAKOŚCI
# =====================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, row_number, floor, collect_list,
    regexp_extract, length
)
from pyspark.sql.window import Window

# 1. SparkSession
spark = SparkSession.builder.appName("FASTQ_Validation_And_Quality").getOrCreate()

# 2. Ścieżka do plików FASTQ
fastq_dir = "/Volumes/databrics_2/default/fastq"
files = ["SRR16356246_1.fastq", "SRR16356246_2.fastq"]
paths = [f"{fastq_dir}/{file}" for file in files]

# 3. Wczytanie plików jako tekst
raw_df = spark.read.text(paths)

# 4. Numerowanie wierszy
window = Window.orderBy("value")  # brak partitionBy → ostrzeżenie na dużych plikach
df_indexed = raw_df.withColumn("row_num", row_number().over(window))

# 5. Grupowanie co 4 linie
df_indexed = df_indexed.withColumn("group_id", floor((col("row_num") - 1) / 4))

fastq_grouped = df_indexed.groupBy("group_id") \
    .agg(collect_list("value").alias("lines"))

# 6. Mapowanie na kolumny FASTQ
fastq_df = fastq_grouped.select(
    col("lines").getItem(0).alias("header"),
    col("lines").getItem(1).alias("sequence"),
    col("lines").getItem(2).alias("plus"),
    col("lines").getItem(3).alias("quality")
)

fastq_df.show(5, truncate=False)

# =====================================================
# ZADANIE 1: WALIDACJA DŁUGOŚCI
# =====================================================
df_with_lengths = fastq_df.withColumn(
    "declared_length",
    regexp_extract(col("header"), r"length=(\d+)", 1).cast("int")
).withColumn(
    "actual_length",
    length(col("sequence"))
)

invalid_records = df_with_lengths.filter(
    col("declared_length") != col("actual_length")
)

invalid_count = invalid_records.count()
print("Liczba niespójnych rekordów:", invalid_count)

# =====================================================
# ZADANIE 2: WSTĘPNA ANALIZA JAKOŚCI
# =====================================================
df_quality = fastq_df.withColumn(
    "low_quality",
    col("quality").contains("#")
)

low_quality_reads = df_quality.filter(col("low_quality") == True)

low_quality_count = low_quality_reads.count()
print("Liczba odczytów o bardzo niskiej jakości (znak '#'):", low_quality_count)

# =====================================================
# KOMENTARZE ANALITYCZNE
# =====================================================

# Zadanie 1:
# - Operacje regexp_extract + length są droższe obliczeniowo
# - Liczba Jobów = 1 (akacja .count())
# - Stage’y: narrow transformations, brak shuffle → 1 Stage
# - Taski = liczba partycji

# Zadanie 2:
# - Operacja .contains("#") jest bardzo szybka
# - Jeśli fastq_df było cache'owane, Stage może mieć etykietę "skipped"
# - Locality Level tasków = PROCESS_LOCAL (dane w pamięci)
# - Liczba Records/task ≈ liczba rekordów w partycji


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+------+--------+----+-------+
|header|sequence|plus|quality|
+------+--------+----+-------+
+------+--------+----+-------+

Liczba niespójnych rekordów: 0
Liczba odczytów o bardzo niskiej jakości (znak '#'): 0
